# [Question 1] Looking back at Scratch

In [2]:
"""
TensorFlow Task
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()


dataset_path ="../data/Iris.csv"
df = pd.read_csv(dataset_path)


df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int32)[:, np.newaxis]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# 【問題2】スクラッチとTensorFlowの対応を考える

In [6]:
#get mini batch 
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int32)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]


#hyperparameters 
learning_rate = 0.001
batch_size = 8
num_epochs = 100

n_hidden1 = 64
n_hidden2 = 128
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# tensorflow array 
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

## dataloader 
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# model architecture
def example_net(x):

    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# model with initialized parameters                       
logits = example_net(X)


#loss function 
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

# optimizer 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

## accuracy 
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

print("Not trained model's accuracy:", accuracy)


### initialize training process
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int32)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Not trained model's accuracy: Tensor("Mean_5:0", shape=(), dtype=float32)
Epoch 0, loss : 191.4909, val_loss : 115.4450, acc : 0.375, val_acc : 0.625
Epoch 1, loss : 50.6023, val_loss : 34.5704, acc : 0.375, val_acc : 0.625
Epoch 2, loss : 22.3705, val_loss : 33.5257, acc : 0.625, val_acc : 0.375
Epoch 3, loss : 12.3241, val_loss : 17.9375, acc : 0.625, val_acc : 0.375
Epoch 4, loss : 5.3521, val_loss : 9.6699, acc : 0.500, val_acc : 0.625
Epoch 5, loss : 1.2759, val_loss : 3.3070, acc : 0.875, val_acc : 0.625
Epoch 6, loss : 0.0000, val_loss : 2.5957, acc : 1.000, val_acc : 0.750
Epoch 7, loss : 0.0000, val_loss : 1.9192, acc : 1.000, val_acc : 0.812
Epoch 8, loss : 0.0000, val_loss : 1.5111, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0000, val_loss : 1.6852, acc : 1.000, val_acc : 0.750
Epoch 10, loss : 0.0000, val_loss : 1.2861, acc : 1.000, val_acc : 0.875
Epoch 11, loss : 0.0000, val_loss : 1.2948, acc : 1.000, val_acc : 0.875
Epoch 12, loss : 0.0000, val_loss : 1.2084, acc : 

# [Problem 3] Create a model for Iris using all three objective variables

In [7]:
from sklearn.preprocessing import OneHotEncoder

dataset_path ="../data/Iris.csv"
df = pd.read_csv(dataset_path)

y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y[y=='Iris-setosa'] = 2

enc = OneHotEncoder(handle_unknown='ignore')
y = enc.fit_transform(y[:,np.newaxis]).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [8]:
y_train.shape

(96, 3)

In [10]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int32)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [12]:
learning_rate = 0.01
batch_size = 8
num_epochs = 100

n_hidden1 = 64
n_hidden2 = 128
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

   
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

#model initiliaze                            
logits = example_net(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))

#not trained model's accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## training 
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int32)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 34.5489, val_loss : 20.2011, acc : 0.500, val_acc : 0.708
Epoch 1, loss : 2.5443, val_loss : 2.7393, acc : 0.750, val_acc : 0.750
Epoch 2, loss : 0.2741, val_loss : 2.1162, acc : 0.875, val_acc : 0.792
Epoch 3, loss : 1.3726, val_loss : 7.5848, acc : 0.750, val_acc : 0.708
Epoch 4, loss : 0.0677, val_loss : 4.9115, acc : 1.000, val_acc : 0.750
Epoch 5, loss : 0.0493, val_loss : 1.1157, acc : 1.000, val_acc : 0.917
Epoch 6, loss : 0.0014, val_loss : 1.2971, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0000, val_loss : 1.5566, acc : 1.000, val_acc : 0.917
Epoch 8, loss : 0.9401, val_loss : 1.1744, acc : 0.875, val_acc : 0.833
Epoch 9, loss : 0.0000, val_loss : 2.8114, acc : 1.000, val_acc : 0.833
Epoch 10, loss : 0.0000, val_loss : 1.7293, acc : 1.000, val_acc : 0.917
Epoch 11, loss : 0.0000, val_loss : 1.8355, acc : 1.000, val_acc : 0.917
Epoch 12, loss : 0.0000, val_loss : 2.2142, acc : 1.000, val_acc : 0.917
Epoch 13, loss : 0.0000, val_loss : 1.9732, acc : 1.000, va

# 【問題4】House Pricesのモデルを作成

In [13]:
import pandas as pd
import numpy as np
data_ori = pd.read_csv("../data/house_price/train.csv")

In [14]:
max_num_of_row = len(data_ori)
data_loss_rate = data_ori.isnull().sum() / max_num_of_row * 100

In [15]:
data_drop_5nan_col = data_ori.dropna(axis=1, thresh=max_num_of_row-5)
print("before:{}".format(len(data_ori.columns.values)))
print("after:{}".format(len(data_drop_5nan_col.columns.values)))

before:81
after:63


In [16]:
data_to_use = data_drop_5nan_col.dropna(axis=0)
data_to_use = data_to_use.reset_index(drop=True)

In [17]:
col_featuer = ['OverallQual',
               'GrLivArea',
               'GarageCars',
               'GarageArea',
               'TotalBsmtSF',
               '1stFlrSF',
               'FullBath',
               'TotRmsAbvGrd',
               'YearBuilt',
               'YearRemodAdd',
              ]

col_target = ['SalePrice']

cols = col_featuer + col_target

In [18]:
data = data_to_use[cols]

In [19]:
import gc 
gc.enable()
gc.collect()

1846

In [20]:
del data_ori, data_drop_5nan_col, data_to_use
gc.collect()

0

In [21]:
X = data[col_featuer].values
Y = data[col_target].values

In [23]:
# data split to train, validation and test 
# train for model training 
# validation for model selection
# test for model evaluation 
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [25]:
# hyperparameters
learning_rate = 0.01
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = x_train.shape[1]
n_samples = x_train.shape[0]
n_classes = 1

# tf tensors
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# dataloader 
get_mini_batch_train = GetMiniBatch(x_train, y_train, batch_size=batch_size)

# model 
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# not trained model's prediction                             
pred = example_net(X)

# loss for regression 
# mean absolute loss is good for outliers 
loss_op = tf.reduce_mean(tf.abs(Y - pred))

# optimizer 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# evaluation metric
r_square = 1 - (tf.reduce_sum(tf.square(Y - pred)) / tf.reduce_sum(tf.square(Y - tf.reduce_mean(Y))))

#training 
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int32)
        total_loss = 0
        total_r_square = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, r_2 = sess.run([loss_op, r_square], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_r_square += r_2
        total_loss /= n_samples
        total_r_square /= n_samples
        val_loss, val_r_square = sess.run([loss_op, r_square], feed_dict={X: x_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, r_square : {:.3f}, val_r_square : {:.3f}".format(epoch, loss, val_loss, r_2, val_r_square))
    test_r_square= sess.run(r_square, feed_dict={X: x_test, Y: y_test})
    print("test_r_square : {:.3f}".format(test_r_square))

Epoch 0, loss : 22132.5312, val_loss : 39249.6289, r_square : 0.964, val_r_square : 0.201
Epoch 1, loss : 17870.1973, val_loss : 41958.8750, r_square : 0.972, val_r_square : 0.145
Epoch 2, loss : 20267.5625, val_loss : 40646.4023, r_square : 0.974, val_r_square : 0.144
Epoch 3, loss : 20940.0312, val_loss : 37188.8789, r_square : 0.974, val_r_square : 0.190
Epoch 4, loss : 19285.4238, val_loss : 32426.0039, r_square : 0.975, val_r_square : 0.215
Epoch 5, loss : 20636.6660, val_loss : 32761.6387, r_square : 0.975, val_r_square : 0.208
Epoch 6, loss : 21218.2812, val_loss : 33296.0625, r_square : 0.974, val_r_square : 0.262
Epoch 7, loss : 21640.4121, val_loss : 32931.2852, r_square : 0.974, val_r_square : 0.275
Epoch 8, loss : 19406.0098, val_loss : 32657.3125, r_square : 0.973, val_r_square : 0.239
Epoch 9, loss : 24951.7598, val_loss : 29763.9102, r_square : 0.966, val_r_square : 0.309
Epoch 10, loss : 26955.1309, val_loss : 37164.0273, r_square : 0.965, val_r_square : 0.112
Epoch 11,

# 【Task5】MNIST Dataset

In [26]:
from keras.datasets import mnist

# MNIST dataset 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train /= 255
x_test /= 255
enc = OneHotEncoder(handle_unknown='ignore')
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis]).toarray()
y_test_one_hot = enc.transform(y_test[:, np.newaxis]).toarray()

In [27]:
#train , val
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train_one_hot, test_size=0.4, shuffle=True, random_state=42)
print(x_train.shape) 
print(x_val.shape) 

(36000, 784)
(24000, 784)


In [29]:
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = x_train.shape[1]
n_samples = x_train.shape[0]
n_classes = 10

#tf tensor
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

#dataloader
get_mini_batch_train = GetMiniBatch(x_train, y_train, batch_size=batch_size)

# model architecture
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    return layer_output

#model 
logits = example_net(X)

# loss 
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

#optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#not trained model's result
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))
#evaluation metric 
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#model training 
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int32)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: x_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: x_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.7149, val_loss : 1.2107, acc : 0.700, val_acc : 0.642
Epoch 1, loss : 0.8950, val_loss : 1.0005, acc : 0.800, val_acc : 0.741
Epoch 2, loss : 0.4705, val_loss : 0.7974, acc : 0.900, val_acc : 0.796
Epoch 3, loss : 0.4889, val_loss : 0.6356, acc : 0.900, val_acc : 0.822
Epoch 4, loss : 0.0649, val_loss : 0.4674, acc : 1.000, val_acc : 0.889
Epoch 5, loss : 0.0454, val_loss : 0.4833, acc : 1.000, val_acc : 0.904
Epoch 6, loss : 0.2041, val_loss : 0.4245, acc : 1.000, val_acc : 0.900
Epoch 7, loss : 0.0307, val_loss : 0.3914, acc : 1.000, val_acc : 0.918
Epoch 8, loss : 0.0374, val_loss : 0.3476, acc : 1.000, val_acc : 0.922
Epoch 9, loss : 0.0252, val_loss : 0.3483, acc : 1.000, val_acc : 0.927
test_acc : 0.930
